In [ ]:
import requests
import datetime
import pandas as pd
import os
from multiprocessing import Pool



In [11]:
import json

# List of cities you are interested in and their corresponding OpenWeatherMap city names
city_names = {
    'Anchorage': 'Anchorage',
    'Boise': 'Boise',
    'Chicago': 'Chicago',
    'Denver': 'Denver',
    'Detroit': 'Detroit',
    'Honolulu': 'Honolulu',
    'Houston': 'Houston',
    'Miami': 'Miami',
    'Minneapolis': 'Minneapolis',
    'Oklahoma City': 'Oklahoma City',
    'Nashville': 'Nashville',
    'New York': 'New York',
    'Phoenix': 'Phoenix',
    'Portland ME': 'Portland',
    'Portland OR': 'Portland',
    'Salt Lake City': 'Salt Lake City',
    'San Diego': 'San Diego',
    'San Francisco': 'San Francisco',
    'Seattle': 'Seattle',
    'Washington DC': 'Washington'
}

# Load the JSON data
with open('../data/openweather/current.city.list.json', 'r', encoding='utf-8') as f:
    city_data = json.load(f)

# Extract city information (adjusted for the JSON structure provided)
results = {}
for city in city_data:
    city_name = city['name']
    # Check if the city is in our list, and handle Portland distinction by country code and other attributes if available
    if city_name in city_names.values():
        if city_name == 'Portland':
            # Distinguish Portland ME from Portland OR (assuming US country and possible state data available)
            if city['country'] == 'US':
                # Assuming 'stat' key or any additional data to differentiate
                if (city_names['Portland ME'] == city_name and city.get('stat', {}).get('population', 0) < 100000) or \
                   (city_names['Portland OR'] == city_name and city.get('stat', {}).get('population', 0) >= 100000):
                    results[city['name']] = city.get('coord', {}).get('lat', 'N/A'), city.get('coord', {}).get('lon', 'N/A')
        else:
            results[city['name']] = city.get('coord', {}).get('lat', 'N/A'), city.get('coord', {}).get('lon', 'N/A')

# Print results
for city, coords in results.items():
    print(f"{city}: Latitude {coords[0]}, Longitude {coords[1]}")


San Francisco: Latitude 37.774929, Longitude -122.419418
Washington: Latitude 47.500118, Longitude -120.501472
San Diego: Latitude 32.715328, Longitude -117.157257
Miami: Latitude 25.774269, Longitude -80.193657
Oklahoma City: Latitude 35.46756, Longitude -97.516434
Nashville: Latitude 36.16589, Longitude -86.784439
Houston: Latitude 29.763281, Longitude -95.363274
Portland: Latitude 45.523449, Longitude -122.676208
Chicago: Latitude 41.850029, Longitude -87.650047
Detroit: Latitude 42.331429, Longitude -83.045753
Minneapolis: Latitude 44.979969, Longitude -93.26384
New York: Latitude 43.000351, Longitude -75.499901
Phoenix: Latitude 33.44838, Longitude -112.074043
Denver: Latitude 39.739151, Longitude -104.984703
Boise: Latitude 43.613499, Longitude -116.203453
Salt Lake City: Latitude 40.76078, Longitude -111.891052
Seattle: Latitude 47.606209, Longitude -122.332069
Honolulu: Latitude 21.30694, Longitude -157.858337
Anchorage: Latitude 61.21806, Longitude -149.900284


In [33]:
import requests

# Your OpenWeatherMap API key
API_KEY = '59016685e15f5d93325917ec6e2e1fb1'  # Replace with your actual API key

# Example latitude and longitude
lat = 36.16589
lon = -86.784439

# Base URL for OpenWeatherMap's Current Weather API
url = "https://api.openweathermap.org/data/2.5/weather?lat=36.16&lon=-86.78&appid=d817ee55d93f6b51d9b15bc850821726&units=imperial"

# Make the API request
response = requests.get(url)

# Check response status
if response.status_code == 200:
    weather_data = response.json()
    print("Current Weather Data:")
    print(weather_data)
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")


Current Weather Data:
{'coord': {'lon': -86.78, 'lat': 36.16}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 60.75, 'feels_like': 59.67, 'temp_min': 57.09, 'temp_max': 61.92, 'pressure': 1015, 'humidity': 67, 'sea_level': 1015, 'grnd_level': 993}, 'visibility': 10000, 'wind': {'speed': 10.36, 'deg': 140}, 'clouds': {'all': 0}, 'dt': 1731549199, 'sys': {'type': 1, 'id': 4609, 'country': 'US', 'sunrise': 1731500552, 'sunset': 1731537638}, 'timezone': -21600, 'id': 4644585, 'name': 'Nashville', 'cod': 200}


In [34]:
import requests
import csv
import os

# Your OpenWeatherMap API key
API_KEY = 'd817ee55d93f6b51d9b15bc850821726'  # Replace with your actual API key

# Dictionary containing city names and their coordinates (latitude, longitude)
city_coordinates = {
    'Anchorage': (61.2181, -149.9003),
    'Boise': (43.615, -116.2023),
    'Chicago': (41.8781, -87.6298),
    'Denver': (39.7392, -104.9903),
    'Detroit': (42.3314, -83.0458),
    'Honolulu': (21.3069, -157.8583),
    'Houston': (29.7604, -95.3698),
    'Miami': (25.7617, -80.1918),
    'Minneapolis': (44.9778, -93.265),
    'Oklahoma City': (35.4676, -97.5164),
    'Nashville': (36.1627, -86.7816),
    'New York': (40.7128, -74.006),
    'Phoenix': (33.4484, -112.074),
    'Portland ME': (43.6591, -70.2568),
    'Portland OR': (45.5051, -122.675),
    'Salt Lake City': (40.7608, -111.891),
    'San Diego': (32.7157, -117.1611),
    'San Francisco': (37.7749, -122.4194),
    'Seattle': (47.6062, -122.3321),
    'Washington DC': (38.9072, -77.0369)
}

BASE_URL = 'https://api.openweathermap.org/data/2.5/weather'

# Directory to save the combined weather data file
output_dir = '../data/openweather/current'
os.makedirs(output_dir, exist_ok=True)

# CSV file path
csv_file_path = os.path.join(output_dir, "detailed_weather_data.csv")

# Fetch weather data and save to CSV
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow([
        "City", "Latitude", "Longitude", "Weather Description", "Temperature (F)", "Feels Like (F)",
        "Temp Min (F)", "Temp Max (F)", "Pressure", "Humidity", "Sea Level", "Ground Level",
        "Wind Speed", "Wind Deg", "Wind Gust", "Clouds All", "Datetime", "Country",
        "Sunrise", "Sunset", "Timezone", "City ID", "City Name"
    ])

    # Function to fetch weather data for a city
    def fetch_weather(city, lat, lon):
        params = {
            'lat': lat,
            'lon': lon,
            'appid': API_KEY,
            'units': 'imperial'  # Use 'metric' for Celsius
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            weather_data = response.json()
            # Extract relevant data
            coord = weather_data.get('coord', {})
            weather = weather_data.get('weather', [{}])[0]
            main = weather_data.get('main', {})
            wind = weather_data.get('wind', {})
            clouds = weather_data.get('clouds', {})
            sys = weather_data.get('sys', {})

            # Create a row with all requested data, handling missing values
            return [
                city,
                coord.get('lat', 'N/A'), coord.get('lon', 'N/A'),
                weather.get('description', 'N/A'),
                main.get('temp', 'N/A'), main.get('feels_like', 'N/A'),
                main.get('temp_min', 'N/A'), main.get('temp_max', 'N/A'),
                main.get('pressure', 'N/A'), main.get('humidity', 'N/A'),
                main.get('sea_level', 'N/A'), main.get('grnd_level', 'N/A'),
                wind.get('speed', 'N/A'), wind.get('deg', 'N/A'), wind.get('gust', 'N/A'),
                clouds.get('all', 'N/A'),
                weather_data.get('dt', 'N/A'), sys.get('country', 'N/A'),
                sys.get('sunrise', 'N/A'), sys.get('sunset', 'N/A'),
                weather_data.get('timezone', 'N/A'), weather_data.get('id', 'N/A'),
                weather_data.get('name', 'N/A')
            ]
        else:
            print(f"Error fetching data for {city}. Status code: {response.status_code}")
            return [city] + ["Error"] * 21

    # Fetch and write weather data for each city
    for city, (lat, lon) in city_coordinates.items():
        city_weather = fetch_weather(city, lat, lon)
        writer.writerow(city_weather)

print(f"Detailed weather data saved to {csv_file_path}")

Detailed weather data saved to ../data/openweather/current/detailed_weather_data.csv


In [36]:
 import pandas as pd
 combined = pd.read_csv("../data/openweather/current/detailed_weather_data.csv")
 combined.head(10)

,City,Latitude,Longitude,Weather Description,Temperature (F),Feels Like (F),Temp Min (F),Temp Max (F),Pressure,Humidity,...,Wind Deg,Wind Gust,Clouds All,Datetime,Country,Sunrise,Sunset,Timezone,City ID,City Name
0,Anchorage,61.2181,-149.9003,mist,16.00,16.00,12.63,18.19,1007,90,...,0,NaN,75,1731549303,US,1731520923,1731547568,-32400,5879400,Anchorage
1,Boise,43.6166,-116.2009,overcast clouds,50.92,47.64,47.35,52.99,1011,41,...,110,27.63,100,1731549121,US,1731508624,1731543690,-25200,5586437,Boise
2,Chicago,41.8781,-87.6298,mist,48.40,42.35,47.08,49.08,1012,89,...,124,20.00,100,1731548947,US,1731501504,1731537093,-21600,4887398,Chicago
3,Denver,39.7420,-104.9915,overcast clouds,39.33,39.33,36.64,41.54,1017,64,...,285,0.74,100,1731549332,US,1731505380,1731541553,-25200,5419384,Denver
4,Detroit,42.3316,-83.0466,clear sky,50.27,48.20,48.00,52.07,1019,68,...,110,18.41,0,1731549340,US,1731500469,1731535929,-18000,4990729,Detroit
5,Honolulu,21.3069,-157.8583,scattered clouds,81.16,84.22,78.89,82.22,1018,66,...,60,28.77,40,1731548920,US,1731516096,1731556215,-36000,5856195,Honolulu
6,Houston,29.7589,-95.3677,clear sky,72.75,73.11,69.91,75.27,1016,72,...,13,4.00,0,1731549219,US,1731501902,1731540410,-21600,4699066,Houston
7,Miami,25.7617,-80.1918,scattered clouds,77.63,78.57,76.01,79.02,1017,74,...,80,29.93,40,1731549421,US,1731497860,1731537167,-18000,4164138,Miami
8,Minneapolis,44.9800,-93.2700,mist,42.91,40.17,41.40,44.80,1013,95,...,130,NaN,100,1731549362,US,1731503327,1731537978,-21600,5037649,Minneapolis
9,Oklahoma City,35.4730,-97.5171,clear sky,54.46,52.29,50.59,55.72,1019,57,...,300,NaN,0,1731549144,US,1731503049,1731540295,-21600,4544349,Oklahoma City
